In [1]:
import pandas as pd
import numpy as np
import torch
import json
import os
from Network.deepnetwork2 import ImitationNetwork
from Memory.dataset import *
from torch.utils.data import DataLoader
from Utils.utils import *
from sklearn.metrics import precision_score, recall_score
import torch.nn as nn
from ImitationAgent import *
from Config.config import *

In [13]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('TensorBoard/1stImitationAgent_AllActions_OldData')


In [3]:
len(decode_state_old_test(a))

516

In [2]:
a = {'player_id': 1, 'player_life': 39, 'player_gold': 0, 'opponent_life': 39, 'time': 59.839996337890625, 'player_board_card_info': [{'id': 23, 'status': 1, 'damage': 1, 'life': 0}, {'id': 7, 'status': 2, 'damage': 1, 'life': 2}, {'id': 50, 'status': 2, 'damage': 2, 'life': 1}, {'id': 50, 'status': 2, 'damage': 
2, 'life': 1}, {'id': 31, 'status': 2, 'damage': 1, 'life': 1}, {'id': 20, 'status': 0, 'damage': 3, 'life': 3}, {'id': 0, 'status': 0, 'damage': 0, 'life': 0}], 'opponent_board_card_info': [{'id': 23, 'status': 0, 'damage': 1, 'life': 2}, {'id': 31, 'status': 0, 'damage': 1, 'life': 0}, {'id': 7, 'status': 2, 'damage': 1, 'life': 2}, {'id': 34, 'status': 0, 'damage': 2, 'life': 1}, {'id': 0, 'status': 0, 'damage': 0, 'life': 0}, {'id': 0, 'status': 0, 'damage': 0, 'life': 0}, {'id': 0, 'status': 0, 'damage': 0, 'life': 0}], 'player_hand_card_id': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'player_remaining_cards_on_deck': [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1], 'opponent_remaining_cards': [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 
1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'player_used_cards': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 
0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'opponent_used_cards': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],'action':21}

In [4]:
dataset = Feeder3('Log/ImitationLog/Train/')

In [6]:
dataloader = DataLoader(dataset,batch_size = 256, num_workers = 20,shuffle=True)

In [6]:
def accuracy(pred, label):
    pred = torch.argmax(pred, dim=1).long()
#     print(pred)
    label = torch.argmax(label,dim=1).long()
    acc = torch.mean((pred == label).float())
    pred = pred.detach().cpu().numpy()
    label = label.detach().cpu().numpy()
    p = precision_score(label, pred,average='micro')
    r = recall_score(label, pred,average='micro')
    return p,r,acc 

class AverageMeter(object):

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [4]:
model = ImitationNetwork(0.1,6,'Test',516,'Test')
model.to(torch.device('cuda:0'))

ImitationNetwork(
  (dense1): Linear(in_features=516, out_features=2048, bias=True)
  (dense2): Linear(in_features=2048, out_features=1024, bias=True)
  (dense3): Linear(in_features=1024, out_features=512, bias=True)
  (dense4): Linear(in_features=512, out_features=256, bias=True)
  (dense5): Linear(in_features=256, out_features=128, bias=True)
  (dense6): Linear(in_features=128, out_features=64, bias=True)
  (dense7): Linear(in_features=64, out_features=32, bias=True)
  (dense8): Linear(in_features=32, out_features=6, bias=True)
  (loss): CrossEntropyLoss()
)

In [10]:
x = np.array([2,3,4,1,2,8,9])

In [11]:
np.argsort(np.max(x, axis=0))

array([0], dtype=int64)

In [19]:
crit = nn.BCEWithLogitsLoss().to(torch.device('cuda:1'))
losses = 0.0
accs  = 0.0
precisions  = AverageMeter()
recalls  = AverageMeter()

opt = torch.optim.SGD(model.parameters(), 0.01, 
                          momentum=0.9, 
                          weight_decay=1e-4) 
# checkpoint = torch.load('Models/1stImitationAgent_OldData.ckpt')
# model.load_state_dict(checkpoint)
for epoch in range(20):
    for i, (data,reward,label) in enumerate(dataloader):
        data_,reward_,label_ = map(lambda x:x.to(torch.device('cuda:1')),(data,reward,label))

        pred = model.forward(data_.float())
#         y_onehot = label.detach().numpy()
#         y_onehot = (np.arange(56) == y_onehot[:,None]).astype(np.float32)
#         y_onehot = torch.from_numpy(y_onehot).to(torch.device('cuda:1'))
        loss = crit(pred,torch.nn.functional.softmax(reward_))
        p,r, acc = accuracy(pred, torch.nn.functional.softmax(reward_))

        opt.zero_grad()
        loss.backward()
        opt.step()

#         losses.update(loss.item(),530)
#         accs.update(acc.item(),530)
        losses += loss.item()
        accs += acc.item()
        precisions.update(p, 530)
        recalls.update(r,530)

        if i % 5 == 0:
            print('Epoch:[{0}][{1}/{2}]\t'
#                   'Loss {losses.val:.3f} ({losses.avg:.3f})\t'
#                   'Accuracy {accs.val:.3f} ({accs.avg:.3f})\t'
                  'Loss {losses:.3f} \t'
                  'Accuracy {accs:.3f}\t'
                  'Precison {precisions.val:.3f} ({precisions.avg:.3f})\t'
                  'Recall {recalls.val:.3f} ({recalls.avg:.3f})'.format(
                        epoch,i,len(dataloader) ,losses=loss, accs=acc, 
                        precisions=precisions, recalls=recalls))
        if i % 10 == 0:
            torch.save(model.state_dict(), 'Models/1stImitationAgent_OldData.ckpt')
            # ...log the running loss
            writer.add_scalar('training loss',
                            losses / 10,
                            epoch * len(dataloader) + i)

            # ...log a Matplotlib Figure showing the model's predictions on a
            # random mini-batch
            writer.add_scalar('accuracy',
                            accs / 10,
                            global_step=epoch * len(dataloader) + i)
            losses = 0.0
            accs = 0.0
    #         print('Epoch:[{0}][{1}/{2}]\t'
    #               'Loss {losses:.3f} \t'
    #               'Accuracy {accs:.3f}\t'
    #               'Precison {precisions:.3f}\t'
    #               'Recall {recalls:.3f}'.format(
    #                     epoch, losses=loss, accs=acc, 
    #                     precisions=p, recalls=r))
    

/home/kietcdx/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/kietcdx/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch:[0][0/1224]	Loss 0.671 	Accuracy 0.180	Precison 0.180 (0.180)	Recall 0.180 (0.180)
Epoch:[0][5/1224]	Loss 0.668 	Accuracy 0.172	Precison 0.172 (0.172)	Recall 0.172 (0.172)
Epoch:[0][10/1224]	Loss 0.662 	Accuracy 0.160	Precison 0.160 (0.175)	Recall 0.160 (0.175)
Epoch:[0][15/1224]	Loss 0.651 	Accuracy 0.172	Precison 0.172 (0.174)	Recall 0.172 (0.174)
Epoch:[0][20/1224]	Loss 0.641 	Accuracy 0.145	Precison 0.145 (0.170)	Recall 0.145 (0.170)
Epoch:[0][25/1224]	Loss 0.630 	Accuracy 0.160	Precison 0.160 (0.171)	Recall 0.160 (0.171)
Epoch:[0][30/1224]	Loss 0.617 	Accuracy 0.156	Precison 0.156 (0.171)	Recall 0.156 (0.171)
Epoch:[0][35/1224]	Loss 0.609 	Accuracy 0.176	Precison 0.176 (0.169)	Recall 0.176 (0.169)
Epoch:[0][40/1224]	Loss 0.593 	Accuracy 0.469	Precison 0.469 (0.198)	Recall 0.469 (0.198)
Epoch:[0][45/1224]	Loss 0.582 	Accuracy 0.500	Precison 0.500 (0.226)	Recall 0.500 (0.226)
Epoch:[0][50/1224]	Loss 0.570 	Accuracy 0.477	Precison 0.477 (0.249)	Recall 0.477 (0.249)
Epoch:[0][55

Epoch:[0][845/1224]	Loss 0.373 	Accuracy 0.527	Precison 0.527 (0.450)	Recall 0.527 (0.450)
Epoch:[0][850/1224]	Loss 0.394 	Accuracy 0.480	Precison 0.480 (0.450)	Recall 0.480 (0.450)
Epoch:[0][855/1224]	Loss 0.388 	Accuracy 0.441	Precison 0.441 (0.450)	Recall 0.441 (0.450)
Epoch:[0][860/1224]	Loss 0.368 	Accuracy 0.480	Precison 0.480 (0.450)	Recall 0.480 (0.450)
Epoch:[0][865/1224]	Loss 0.372 	Accuracy 0.555	Precison 0.555 (0.451)	Recall 0.555 (0.451)
Epoch:[0][870/1224]	Loss 0.369 	Accuracy 0.535	Precison 0.535 (0.451)	Recall 0.535 (0.451)
Epoch:[0][875/1224]	Loss 0.379 	Accuracy 0.465	Precison 0.465 (0.451)	Recall 0.465 (0.451)
Epoch:[0][880/1224]	Loss 0.378 	Accuracy 0.496	Precison 0.496 (0.452)	Recall 0.496 (0.452)
Epoch:[0][885/1224]	Loss 0.370 	Accuracy 0.488	Precison 0.488 (0.452)	Recall 0.488 (0.452)
Epoch:[0][890/1224]	Loss 0.372 	Accuracy 0.508	Precison 0.508 (0.452)	Recall 0.508 (0.452)
Epoch:[0][895/1224]	Loss 0.363 	Accuracy 0.539	Precison 0.539 (0.453)	Recall 0.539 (0.453)

Epoch:[1][70/1224]	Loss 0.357 	Accuracy 0.555	Precison 0.555 (0.470)	Recall 0.555 (0.470)
Epoch:[1][75/1224]	Loss 0.364 	Accuracy 0.477	Precison 0.477 (0.471)	Recall 0.477 (0.471)
Epoch:[1][80/1224]	Loss 0.365 	Accuracy 0.523	Precison 0.523 (0.471)	Recall 0.523 (0.471)
Epoch:[1][85/1224]	Loss 0.347 	Accuracy 0.531	Precison 0.531 (0.471)	Recall 0.531 (0.471)
Epoch:[1][90/1224]	Loss 0.366 	Accuracy 0.500	Precison 0.500 (0.471)	Recall 0.500 (0.471)
Epoch:[1][95/1224]	Loss 0.367 	Accuracy 0.543	Precison 0.543 (0.471)	Recall 0.543 (0.471)
Epoch:[1][100/1224]	Loss 0.360 	Accuracy 0.535	Precison 0.535 (0.471)	Recall 0.535 (0.471)
Epoch:[1][105/1224]	Loss 0.360 	Accuracy 0.523	Precison 0.523 (0.471)	Recall 0.523 (0.471)
Epoch:[1][110/1224]	Loss 0.369 	Accuracy 0.523	Precison 0.523 (0.472)	Recall 0.523 (0.472)
Epoch:[1][115/1224]	Loss 0.372 	Accuracy 0.469	Precison 0.469 (0.472)	Recall 0.469 (0.472)
Epoch:[1][120/1224]	Loss 0.370 	Accuracy 0.527	Precison 0.527 (0.472)	Recall 0.527 (0.472)
Epoch

Epoch:[1][525/1224]	Loss 0.362 	Accuracy 0.543	Precison 0.543 (0.483)	Recall 0.543 (0.483)
Epoch:[1][530/1224]	Loss 0.356 	Accuracy 0.527	Precison 0.527 (0.483)	Recall 0.527 (0.483)
Epoch:[1][535/1224]	Loss 0.354 	Accuracy 0.531	Precison 0.531 (0.483)	Recall 0.531 (0.483)
Epoch:[1][540/1224]	Loss 0.345 	Accuracy 0.594	Precison 0.594 (0.484)	Recall 0.594 (0.484)
Epoch:[1][545/1224]	Loss 0.363 	Accuracy 0.527	Precison 0.527 (0.484)	Recall 0.527 (0.484)
Epoch:[1][550/1224]	Loss 0.351 	Accuracy 0.516	Precison 0.516 (0.484)	Recall 0.516 (0.484)
Epoch:[1][555/1224]	Loss 0.370 	Accuracy 0.484	Precison 0.484 (0.484)	Recall 0.484 (0.484)
Epoch:[1][560/1224]	Loss 0.369 	Accuracy 0.535	Precison 0.535 (0.484)	Recall 0.535 (0.484)
Epoch:[1][565/1224]	Loss 0.362 	Accuracy 0.508	Precison 0.508 (0.484)	Recall 0.508 (0.484)
Epoch:[1][570/1224]	Loss 0.347 	Accuracy 0.516	Precison 0.516 (0.484)	Recall 0.516 (0.484)
Epoch:[1][575/1224]	Loss 0.353 	Accuracy 0.520	Precison 0.520 (0.484)	Recall 0.520 (0.484)

...

Epoch:[18][545/1224]	Loss 0.278 	Accuracy 0.875	Precison 0.875 (0.797)	Recall 0.875 (0.797)
Epoch:[18][550/1224]	Loss 0.274 	Accuracy 0.848	Precison 0.848 (0.797)	Recall 0.848 (0.797)
Epoch:[18][555/1224]	Loss 0.294 	Accuracy 0.848	Precison 0.848 (0.797)	Recall 0.848 (0.797)
Epoch:[18][560/1224]	Loss 0.258 	Accuracy 0.891	Precison 0.891 (0.797)	Recall 0.891 (0.797)
Epoch:[18][565/1224]	Loss 0.278 	Accuracy 0.855	Precison 0.855 (0.797)	Recall 0.855 (0.797)
Epoch:[18][570/1224]	Loss 0.286 	Accuracy 0.879	Precison 0.879 (0.797)	Recall 0.879 (0.797)
Epoch:[18][575/1224]	Loss 0.276 	Accuracy 0.902	Precison 0.902 (0.797)	Recall 0.902 (0.797)
Epoch:[18][580/1224]	Loss 0.255 	Accuracy 0.922	Precison 0.922 (0.797)	Recall 0.922 (0.797)
Epoch:[18][585/1224]	Loss 0.264 	Accuracy 0.922	Precison 0.922 (0.797)	Recall 0.922 (0.797)
Epoch:[18][590/1224]	Loss 0.274 	Accuracy 0.922	Precison 0.922 (0.797)	Recall 0.922 (0.797)
Epoch:[18][595/1224]	Loss 0.287 	Accuracy 0.895	Precison 0.895 (0.797)	Recall 0.

Epoch:[18][995/1224]	Loss 0.286 	Accuracy 0.832	Precison 0.832 (0.799)	Recall 0.832 (0.799)
Epoch:[18][1000/1224]	Loss 0.258 	Accuracy 0.910	Precison 0.910 (0.799)	Recall 0.910 (0.799)
Epoch:[18][1005/1224]	Loss 0.289 	Accuracy 0.867	Precison 0.867 (0.799)	Recall 0.867 (0.799)
Epoch:[18][1010/1224]	Loss 0.289 	Accuracy 0.820	Precison 0.820 (0.799)	Recall 0.820 (0.799)
Epoch:[18][1015/1224]	Loss 0.257 	Accuracy 0.922	Precison 0.922 (0.799)	Recall 0.922 (0.799)
Epoch:[18][1020/1224]	Loss 0.269 	Accuracy 0.852	Precison 0.852 (0.799)	Recall 0.852 (0.799)
Epoch:[18][1025/1224]	Loss 0.267 	Accuracy 0.891	Precison 0.891 (0.799)	Recall 0.891 (0.799)
Epoch:[18][1030/1224]	Loss 0.258 	Accuracy 0.930	Precison 0.930 (0.799)	Recall 0.930 (0.799)
Epoch:[18][1035/1224]	Loss 0.277 	Accuracy 0.902	Precison 0.902 (0.799)	Recall 0.902 (0.799)
Epoch:[18][1040/1224]	Loss 0.292 	Accuracy 0.891	Precison 0.891 (0.799)	Recall 0.891 (0.799)
Epoch:[18][1045/1224]	Loss 0.278 	Accuracy 0.871	Precison 0.871 (0.799)

Epoch:[19][215/1224]	Loss 0.262 	Accuracy 0.855	Precison 0.855 (0.801)	Recall 0.855 (0.801)
Epoch:[19][220/1224]	Loss 0.284 	Accuracy 0.902	Precison 0.902 (0.801)	Recall 0.902 (0.801)
Epoch:[19][225/1224]	Loss 0.273 	Accuracy 0.922	Precison 0.922 (0.801)	Recall 0.922 (0.801)
Epoch:[19][230/1224]	Loss 0.259 	Accuracy 0.875	Precison 0.875 (0.801)	Recall 0.875 (0.801)
Epoch:[19][235/1224]	Loss 0.257 	Accuracy 0.902	Precison 0.902 (0.801)	Recall 0.902 (0.801)
Epoch:[19][240/1224]	Loss 0.264 	Accuracy 0.891	Precison 0.891 (0.801)	Recall 0.891 (0.801)
Epoch:[19][245/1224]	Loss 0.274 	Accuracy 0.887	Precison 0.887 (0.801)	Recall 0.887 (0.801)
Epoch:[19][250/1224]	Loss 0.261 	Accuracy 0.879	Precison 0.879 (0.801)	Recall 0.879 (0.801)
Epoch:[19][255/1224]	Loss 0.281 	Accuracy 0.898	Precison 0.898 (0.801)	Recall 0.898 (0.801)
Epoch:[19][260/1224]	Loss 0.276 	Accuracy 0.914	Precison 0.914 (0.801)	Recall 0.914 (0.801)
Epoch:[19][265/1224]	Loss 0.267 	Accuracy 0.891	Precison 0.891 (0.801)	Recall 0.

Epoch:[19][665/1224]	Loss 0.261 	Accuracy 0.930	Precison 0.930 (0.803)	Recall 0.930 (0.803)
Epoch:[19][670/1224]	Loss 0.292 	Accuracy 0.875	Precison 0.875 (0.803)	Recall 0.875 (0.803)
Epoch:[19][675/1224]	Loss 0.279 	Accuracy 0.836	Precison 0.836 (0.803)	Recall 0.836 (0.803)
Epoch:[19][680/1224]	Loss 0.274 	Accuracy 0.898	Precison 0.898 (0.803)	Recall 0.898 (0.803)
Epoch:[19][685/1224]	Loss 0.262 	Accuracy 0.887	Precison 0.887 (0.803)	Recall 0.887 (0.803)
Epoch:[19][690/1224]	Loss 0.261 	Accuracy 0.891	Precison 0.891 (0.803)	Recall 0.891 (0.803)
Epoch:[19][695/1224]	Loss 0.279 	Accuracy 0.926	Precison 0.926 (0.803)	Recall 0.926 (0.803)
Epoch:[19][700/1224]	Loss 0.275 	Accuracy 0.891	Precison 0.891 (0.803)	Recall 0.891 (0.803)
Epoch:[19][705/1224]	Loss 0.247 	Accuracy 0.902	Precison 0.902 (0.803)	Recall 0.902 (0.803)
Epoch:[19][710/1224]	Loss 0.281 	Accuracy 0.926	Precison 0.926 (0.803)	Recall 0.926 (0.803)
Epoch:[19][715/1224]	Loss 0.264 	Accuracy 0.891	Precison 0.891 (0.803)	Recall 0.

Epoch:[19][1120/1224]	Loss 0.279 	Accuracy 0.918	Precison 0.918 (0.805)	Recall 0.918 (0.805)
Epoch:[19][1125/1224]	Loss 0.251 	Accuracy 0.863	Precison 0.863 (0.805)	Recall 0.863 (0.805)
Epoch:[19][1130/1224]	Loss 0.270 	Accuracy 0.910	Precison 0.910 (0.805)	Recall 0.910 (0.805)
Epoch:[19][1135/1224]	Loss 0.267 	Accuracy 0.867	Precison 0.867 (0.805)	Recall 0.867 (0.805)
Epoch:[19][1140/1224]	Loss 0.280 	Accuracy 0.871	Precison 0.871 (0.805)	Recall 0.871 (0.805)
Epoch:[19][1145/1224]	Loss 0.262 	Accuracy 0.918	Precison 0.918 (0.805)	Recall 0.918 (0.805)
Epoch:[19][1150/1224]	Loss 0.265 	Accuracy 0.898	Precison 0.898 (0.805)	Recall 0.898 (0.805)
Epoch:[19][1155/1224]	Loss 0.253 	Accuracy 0.898	Precison 0.898 (0.805)	Recall 0.898 (0.805)
Epoch:[19][1160/1224]	Loss 0.281 	Accuracy 0.898	Precison 0.898 (0.805)	Recall 0.898 (0.805)
Epoch:[19][1165/1224]	Loss 0.252 	Accuracy 0.906	Precison 0.906 (0.805)	Recall 0.906 (0.805)
Epoch:[19][1170/1224]	Loss 0.266 	Accuracy 0.922	Precison 0.922 (0.805

In [3]:
dataset_test = Feeder3('Log/ImitationLog/Test/')
dataloader1 = DataLoader(dataset_test,batch_size = 256, num_workers = 20,shuffle=True)

In [7]:
crit = nn.BCEWithLogitsLoss().to(torch.device('cuda:1'))
losses = AverageMeter()
accs  = AverageMeter()
precisions  = AverageMeter()
recalls  = AverageMeter()

opt = torch.optim.SGD(model.parameters(), 0.01, 
                          momentum=0.9, weight_decay=1e-4) 
checkpoint = torch.load('Models/1stImitationAgent_OldData.ckpt')
model.load_state_dict(checkpoint)
epoch = 0
for i, (data,reward,label) in enumerate(dataloader1):
    data_,reward_,label_ = map(lambda x:x.to(torch.device('cuda:1')),(data,reward,label))

    pred = model.forward(data_.float())
#         y_onehot = label.detach().numpy()
#         y_onehot = (np.arange(56) == y_onehot[:,None]).astype(np.float32)
#         y_onehot = torch.from_numpy(y_onehot).to(torch.device('cuda:1'))
    loss = crit(pred,torch.nn.functional.softmax(reward_))
    p,r, acc = accuracy(pred, torch.nn.functional.softmax(reward_))

#         opt.zero_grad()
#         loss.backward()
#         opt.step()

    losses.update(loss.item(),530)
    accs.update(acc.item(),530)
#     losses += loss.item()
#     accs += acc.item()
    
    precisions.update(p, 530)
    recalls.update(r,530)
    print('Epoch:[{0}][{1}/{2}]\t'
          'Loss {losses.val:.3f} ({losses.avg:.3f})\t'
          'Accuracy {accs.val:.3f} ({accs.avg:.3f})\t'
#           'Loss {losses:.3f} \t'
#           'Accuracy {accs:.3f}\t'
          'Precison {precisions.val:.3f} ({precisions.avg:.3f})\t'
          'Recall {recalls.val:.3f} ({recalls.avg:.3f})'.format(
                epoch,i,len(dataloader1) ,losses=losses, accs=accs, 
                        precisions=precisions, recalls=recalls))

/home/kietcdx/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/kietcdx/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch:[0][0/333]	Loss 0.318 (0.318)	Accuracy 0.906 (0.906)	Precison 0.906 (0.906)	Recall 0.906 (0.906)
Epoch:[0][1/333]	Loss 0.290 (0.304)	Accuracy 0.898 (0.902)	Precison 0.898 (0.902)	Recall 0.898 (0.902)
Epoch:[0][2/333]	Loss 0.308 (0.305)	Accuracy 0.902 (0.902)	Precison 0.902 (0.902)	Recall 0.902 (0.902)
Epoch:[0][3/333]	Loss 0.294 (0.302)	Accuracy 0.926 (0.908)	Precison 0.926 (0.908)	Recall 0.926 (0.908)
Epoch:[0][4/333]	Loss 0.294 (0.301)	Accuracy 0.871 (0.901)	Precison 0.871 (0.901)	Recall 0.871 (0.901)
Epoch:[0][5/333]	Loss 0.313 (0.303)	Accuracy 0.922 (0.904)	Precison 0.922 (0.904)	Recall 0.922 (0.904)
Epoch:[0][6/333]	Loss 0.293 (0.301)	Accuracy 0.906 (0.905)	Precison 0.906 (0.905)	Recall 0.906 (0.905)
Epoch:[0][7/333]	Loss 0.290 (0.300)	Accuracy 0.922 (0.907)	Precison 0.922 (0.907)	Recall 0.922 (0.907)
Epoch:[0][8/333]	Loss 0.302 (0.300)	Accuracy 0.891 (0.905)	Precison 0.891 (0.905)	Recall 0.891 (0.905)
Epoch:[0][9/333]	Loss 0.324 (0.302)	Accuracy 0.902 (0.905)	Precison 0.902

Epoch:[0][80/333]	Loss 0.300 (0.308)	Accuracy 0.938 (0.906)	Precison 0.938 (0.906)	Recall 0.938 (0.906)
Epoch:[0][81/333]	Loss 0.283 (0.307)	Accuracy 0.926 (0.906)	Precison 0.926 (0.906)	Recall 0.926 (0.906)
Epoch:[0][82/333]	Loss 0.278 (0.307)	Accuracy 0.891 (0.906)	Precison 0.891 (0.906)	Recall 0.891 (0.906)
Epoch:[0][83/333]	Loss 0.291 (0.307)	Accuracy 0.910 (0.906)	Precison 0.910 (0.906)	Recall 0.910 (0.906)
Epoch:[0][84/333]	Loss 0.305 (0.307)	Accuracy 0.914 (0.906)	Precison 0.914 (0.906)	Recall 0.914 (0.906)
Epoch:[0][85/333]	Loss 0.283 (0.307)	Accuracy 0.914 (0.906)	Precison 0.914 (0.906)	Recall 0.914 (0.906)
Epoch:[0][86/333]	Loss 0.299 (0.306)	Accuracy 0.906 (0.906)	Precison 0.906 (0.906)	Recall 0.906 (0.906)
Epoch:[0][87/333]	Loss 0.311 (0.307)	Accuracy 0.906 (0.906)	Precison 0.906 (0.906)	Recall 0.906 (0.906)
Epoch:[0][88/333]	Loss 0.297 (0.306)	Accuracy 0.902 (0.906)	Precison 0.902 (0.906)	Recall 0.902 (0.906)
Epoch:[0][89/333]	Loss 0.287 (0.306)	Accuracy 0.914 (0.906)	Prec

Epoch:[0][160/333]	Loss 0.321 (0.306)	Accuracy 0.910 (0.908)	Precison 0.910 (0.908)	Recall 0.910 (0.908)
Epoch:[0][161/333]	Loss 0.308 (0.306)	Accuracy 0.914 (0.908)	Precison 0.914 (0.908)	Recall 0.914 (0.908)
Epoch:[0][162/333]	Loss 0.323 (0.306)	Accuracy 0.934 (0.908)	Precison 0.934 (0.908)	Recall 0.934 (0.908)
Epoch:[0][163/333]	Loss 0.310 (0.306)	Accuracy 0.906 (0.908)	Precison 0.906 (0.908)	Recall 0.906 (0.908)
Epoch:[0][164/333]	Loss 0.291 (0.306)	Accuracy 0.930 (0.908)	Precison 0.930 (0.908)	Recall 0.930 (0.908)
Epoch:[0][165/333]	Loss 0.310 (0.306)	Accuracy 0.902 (0.908)	Precison 0.902 (0.908)	Recall 0.902 (0.908)
Epoch:[0][166/333]	Loss 0.300 (0.306)	Accuracy 0.887 (0.908)	Precison 0.887 (0.908)	Recall 0.887 (0.908)
Epoch:[0][167/333]	Loss 0.303 (0.306)	Accuracy 0.922 (0.908)	Precison 0.922 (0.908)	Recall 0.922 (0.908)
Epoch:[0][168/333]	Loss 0.312 (0.306)	Accuracy 0.902 (0.908)	Precison 0.902 (0.908)	Recall 0.902 (0.908)
Epoch:[0][169/333]	Loss 0.301 (0.306)	Accuracy 0.895 (0

Epoch:[0][240/333]	Loss 0.317 (0.306)	Accuracy 0.934 (0.908)	Precison 0.934 (0.908)	Recall 0.934 (0.908)
Epoch:[0][241/333]	Loss 0.299 (0.306)	Accuracy 0.926 (0.908)	Precison 0.926 (0.908)	Recall 0.926 (0.908)
Epoch:[0][242/333]	Loss 0.308 (0.306)	Accuracy 0.926 (0.908)	Precison 0.926 (0.908)	Recall 0.926 (0.908)
Epoch:[0][243/333]	Loss 0.299 (0.306)	Accuracy 0.938 (0.908)	Precison 0.938 (0.908)	Recall 0.938 (0.908)
Epoch:[0][244/333]	Loss 0.316 (0.306)	Accuracy 0.891 (0.908)	Precison 0.891 (0.908)	Recall 0.891 (0.908)
Epoch:[0][245/333]	Loss 0.312 (0.306)	Accuracy 0.914 (0.908)	Precison 0.914 (0.908)	Recall 0.914 (0.908)
Epoch:[0][246/333]	Loss 0.303 (0.306)	Accuracy 0.918 (0.908)	Precison 0.918 (0.908)	Recall 0.918 (0.908)
Epoch:[0][247/333]	Loss 0.305 (0.306)	Accuracy 0.910 (0.908)	Precison 0.910 (0.908)	Recall 0.910 (0.908)
Epoch:[0][248/333]	Loss 0.291 (0.306)	Accuracy 0.926 (0.908)	Precison 0.926 (0.908)	Recall 0.926 (0.908)
Epoch:[0][249/333]	Loss 0.296 (0.306)	Accuracy 0.906 (0

Epoch:[0][320/333]	Loss 0.293 (0.306)	Accuracy 0.938 (0.908)	Precison 0.938 (0.908)	Recall 0.938 (0.908)
Epoch:[0][321/333]	Loss 0.294 (0.306)	Accuracy 0.930 (0.908)	Precison 0.930 (0.908)	Recall 0.930 (0.908)
Epoch:[0][322/333]	Loss 0.293 (0.306)	Accuracy 0.910 (0.908)	Precison 0.910 (0.908)	Recall 0.910 (0.908)
Epoch:[0][323/333]	Loss 0.301 (0.306)	Accuracy 0.914 (0.908)	Precison 0.914 (0.908)	Recall 0.914 (0.908)
Epoch:[0][324/333]	Loss 0.316 (0.306)	Accuracy 0.879 (0.908)	Precison 0.879 (0.908)	Recall 0.879 (0.908)
Epoch:[0][325/333]	Loss 0.288 (0.306)	Accuracy 0.902 (0.908)	Precison 0.902 (0.908)	Recall 0.902 (0.908)
Epoch:[0][326/333]	Loss 0.285 (0.306)	Accuracy 0.941 (0.908)	Precison 0.941 (0.908)	Recall 0.941 (0.908)
Epoch:[0][327/333]	Loss 0.300 (0.306)	Accuracy 0.902 (0.908)	Precison 0.902 (0.908)	Recall 0.902 (0.908)
Epoch:[0][328/333]	Loss 0.291 (0.306)	Accuracy 0.910 (0.908)	Precison 0.910 (0.908)	Recall 0.910 (0.908)
Epoch:[0][329/333]	Loss 0.307 (0.306)	Accuracy 0.922 (0

In [ ]:
accs

In [ ]:
torch.save(model.state_dict(), 'Models/1stImitationAgent_OldData.ckpt')
# checkpoint = torch.load('Models/1stImitationAgent_OldData.ckpt')
# model.load_state_dict(checkpoint)

In [ ]:
max([1,2,3])

In [ ]:
input

In [ ]:
pred[0]

In [ ]:
f = open('Log/2021-03-11_23-01-35_1.json')
data = json.load(f)

In [ ]:
data = data['logs']

In [ ]:
IMITATION_LOG_DIR = 'Log/'

In [ ]:
a = list()
for i in os.listdir(IMITATION_LOG_DIR):
    f = open(IMITATION_LOG_DIR + i)
    try:
        a.extend(json.load(f)['logs'])
    except json.JSONDecodeError:
        f.close()
        continue
    f.close()

In [ ]:
player_life = [str_env['player_life']]
player_gold = [str_env['player_life']] 
opponent_life = [str_env['opponent_life']]
time = [str_env['time']]
player_board_card_info = list()
opponent_board_card_info = list()

for x in str_env['player_board_card_info']:
    player_board_card_info.append(x['id'])
    player_board_card_info.append(x['status'])
    player_board_card_info.append(x['damage'])
    player_board_card_info.append(x['life'])

for x in str_env['opponent_board_card_info']:
    opponent_board_card_info.append(x['id'])
    opponent_board_card_info.append(x['status'])
    opponent_board_card_info.append(x['damage'])
    opponent_board_card_info.append(x['life'])
    
player_hand_card_id = str_env['player_hand_card_id']

In [ ]:
state = player_life + player_gold + opponent_life + time + player_board_card_info + opponent_board_card_info + player_hand_card_id

In [ ]:
len(state)

In [ ]:
a = torch.Tensor([1,2,2,2])
b = torch.Tensor([1,3,55,3])

In [ ]:
torch.add(a,b)

In [ ]:
test == None

In [ ]:
np.random.choice(1)

In [ ]:
info = pd.read_csv('CardIDList.csv')

In [ ]:
card_info = info[info['ID']==2]

In [ ]:
card_info

In [ ]:
info.at[56,"Name"]